In [135]:
%serialconnect --port=COM7 --baudrate=115200

serial exception on close WriteFile failed (PermissionError(13, 'Access is denied.', None, 5))
Connecting to --port=COM7 --baud=115200 

Ready.


In [136]:
from machine import I2C
import time

In [137]:
i2c = I2C(I2C.I2C0, freq=100000, scl=35, sda=34)

In [143]:
i2c.writeto(0x12,bytes([int(128),int(40),int(0)]))

In [181]:
i2c.writeto(0x12,bytes([int(128),int(210),int(0)]))

In [186]:
i2c.writeto(0x12,bytes([int(160),int(128),int(0)]))

In [190]:
i2c.writeto(0x12,bytes([int(50),int(128),int(0)]))

In [195]:
data = i2c.readfrom(0x12, 3)
print(data)

b'\x80\x80\x04'


In [ ]:
while 1 :
    data = i2c.readfrom(0x12, 3)
    print(data)
    time.sleep(1)

b'\x80\x80\x04'
b'\x80\x80\x04'
b'\x80\x80\x00'
b'\x80\x80\x04'
b'\x80\x80\x00'
b'\x80\x80\x00'
b'\x80\x80\x00'
b'\x80\x80\x04'
b'\x80\x80\x00'
b'\x800\x00'
b'\x800\x00'
b'\x80\xd0\x00'
b'\x800\x00'
b'\x80\xd0\x00'
b'\x800\x00'
b'\x80\x80\x00'
b'h\x80\x00'
b'\x97\x80\x00'
b'h\x80\x00'
b'\x97\x80\x00'
b'\x80\x80\x00'
b'\x80\x80\x00'
b'\x80\x80\x00'
b'\x80\x80\x00'
b'\x80\x80\x00'
b'\x80\x80\x00'
b'\x80\x80\x00'
b'\x80\x80\x00'
b'\x80\x80\x00'
b'\x80\x80\x00'
b'\x80\x80\x00'
b'\x80\x80\x00'
b'\x80\x80\x00'
b'\x80\x80\x00'
b'\x80\x80\x00'
b'\x80\x80\x00'
b'\x80\x80\x00'
b'\x80\x80\x00'
b'\x80\x80\x00'
b'\x80\x80\x00'
b'\x80\x80\x00'
b'\x80\x80\x00'
b'\x80\x80\x00'
b'\x80\x80\x00'
b'\x80\x80\x00'
b'\x80\x80\x00'
b'\x80\x80\x00'
b'\x80\x80\x00'
b'\x80\x80\x00'
b'\x80\x80\x00'
b'\x80\x80\x00'
b'\x80\x80\x00'
b'\x80\x80\x00'
b'\x80\x80\x00'
b'\x80\x80\x00'
b'\x80\x80\x00'
b'\x80\x80\x00'
b'\x80\x80\x00'
b'\x80\x80\x00'
b'\x80\x80\x00'
b'\x80\x80\x00'
b'\x80\x80\x00'
b'\x80\x80\x00'
b'\x80\x80